In [1]:
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statistics
import warnings
from itertools import product

# Suppress all warnings
warnings.filterwarnings('ignore')

In [ ]:
class Data_Storage:
    def __init__(self, ticker, start_date, end_date):
        self.ticker = ticker
        self.start_date = start_date
        self.end_date = end_date
        self.data = yf.download(self.ticker, start=self.start_date, end=self.end_date)
        #self.lookback = self.data[:-]
        self.split_number_when_equally = 1
        

    def manipulate_data(self):
        "Adding columns to the dataframe"
        self.data["Returns"] = self.data['Close'].pct_change()
        self.data["Direction"] = np.where(self.data["Returns"] > 0, "1", "0")
        self.data["weights"] = self.make_weights_splitting()

        return self.data 
    
    
    def make_weights_splitting(self):
        "split_number_when_equally: Equally splits the weights"
        "Eg: 2 splits, is a dataset of 10: 5:2s and 5:1s [2,2,2,2,2,1,1,1,1,1]"
        "    3 splits, in a dataset of 12: 4:3s, 4:2s, 4:1s"

        len_data = len(self.data)
        split = self.split_number_when_equally
        
        repeats = len_data // split
        result = []
        
        for value in range(split, 0, -1):
            result.extend([value] * repeats)

        #cut the end of the df as there is some
        amount_to_cut = len_data - len(result)
        if amount_to_cut > 0:
            result.extend([1] * amount_to_cut)
        return result
    
    
    def set_update_weight_equal_split_number_to_data(self, split_number):
        self.split_number_when_equally = split_number
        self.data["weights"] = self.make_weights_splitting()
        return self.data
    


    
class Get_Incremental_Data:
    def __init__(self, Data_Storage_Object):
        
        self.data = Data_Storage_Object.data
        self.Direction = self.data["Direction"]
        self.Returns = self.data["Returns"]
        self.weights = self.data["weights"]


    def get_direction_increment(self,i):
        return self.Direction.iloc[i]
    
    def get_return_increment(self,i):
        return self.Returns.iloc[i]
    
    def get_weight_increment(self,i):
        return self.weights.iloc[i]



class sliding_window:
    def __init__(self, Data_Storage_Object, Get_Incremental_Data_Object, max_pattern_lenght):

        self.data = Data_Storage_Object.data
        self.Get_Incremental_Data_Object = Get_Incremental_Data_Object
        self.max_pattern_lenght = max_pattern_lenght
        self.Direction = self.data["Direction"]


    def get_lengthed_pattern_at_index(self, i, pattern_lenght):
        #For testing purposes
        #self.Direction.iloc[i-pattern_lenght:i]
        return self.Direction.iloc[i-pattern_lenght+1:i+1].astype(str).str.cat()


    def get_total_weight_and_avg_return_for_all_pattern(self):
        "This will stop at pattern lenght of max_pattern_lenght or if the any pattern has 0 enteries"

        stop_condition = False

        #keepincreasing the pattern_lenght until we see that after checking all patterns of 
        #that lenght that 2^that_pattern_lenght is > then number of recorded pattens of that lenght.

        #Make a dict that records the umber of patterns per lenghts

        #How many patterns we see at each lenght
        dict_pattern_lenght_occurences = {}

        #Main pattern dict
        Main_dict_pattern_info_from_data = {}


        pattern_lenght = 1
        seen_patterns = set()

        while stop_condition == False:
        
            for index in range(pattern_lenght , len(self.data)):
                current_pattern = self.get_lengthed_pattern_at_index(index, pattern_lenght)

                #print(self.Get_Incremental_Data_Object.get_return_increment(index), self.Get_Incremental_Data_Object.get_direction_increment(index), current_pattern)
                


                #Store pattern info in main dict
                if current_pattern not in Main_dict_pattern_info_from_data:
                    Main_dict_pattern_info_from_data[current_pattern] = {
                        "total_avg_return": 0,
                        "total_count_weight": 0,
                        "count": 0
                    }

                #----Get Total Weight and Count-----
                Main_dict_pattern_info_from_data[current_pattern]["total_count_weight"] += self.Get_Incremental_Data_Object.get_weight_increment(index)
                Main_dict_pattern_info_from_data[current_pattern]["count"] += 1

                #----Get Average Return as a Running Average-----
                # Get the new return increment
                new_val = self.Get_Incremental_Data_Object.get_return_increment(index)
                n = Main_dict_pattern_info_from_data[current_pattern]["count"]

                old_avg = Main_dict_pattern_info_from_data[current_pattern].get("total_avg_return", 0)

                # Update the dictionary with the new average
                Main_dict_pattern_info_from_data[current_pattern]["total_avg_return"] = old_avg + (new_val - old_avg) / n


                #count the patter if it has not been seen before
                if current_pattern not in seen_patterns:
                    seen_patterns.add(current_pattern)
                    dict_pattern_lenght_occurences[pattern_lenght] = dict_pattern_lenght_occurences.get(pattern_lenght, 0) + 1


            #Max Patter stop condition
            if 2**pattern_lenght > dict_pattern_lenght_occurences.get(pattern_lenght, 0):
                stop_condition = True
                print(f"Stopping at pattern lenght: {pattern_lenght} as 2^{pattern_lenght} > {dict_pattern_lenght_occurences.get(pattern_lenght, 0)}")
            
            
            pattern_lenght += 1

        return Main_dict_pattern_info_from_data
    





    #-----Make predictions per pattern-----
    def getting_predictions_per_pattern_prob_return_dict(self, Main_dict_pattern_info_from_data):
        "Get the predictions per pattern in  dict"
        pattern_prediction_dict = {}
        for pattern, info in Main_dict_pattern_info_from_data.items():
            total_count_weight = info["total_count_weight"]
            avg_return = info["total_avg_return"]

            

    

    ##PUT THIS DISCT IN THE DATA STORER OBJECT AND CALL THIS FUNCTION/CLASS FROM THERE


    
   

    

In [3]:
dataInst = Data_Storage('AAPL', '2020-01-01', '2023-01-01')
dataInst.manipulate_data()

[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume,Returns,Direction,weights
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,,,
Date,,,,,,,,
2020-01-02,72.468269,72.528589,71.223267,71.476607,135480400,NaN,0,1
2020-01-03,71.763718,72.523746,71.539330,71.696160,146322800,-0.009722,0,1
2020-01-06,72.335556,72.374162,70.634539,70.885472,118387200,0.007968,1,1
2020-01-07,71.995361,72.600968,71.775796,72.345212,108872000,-0.004703,0,1
2020-01-08,73.153496,73.455095,71.698581,71.698581,132079200,0.016086,1,1
...,...,...,...,...,...,...,...,...
2022-12-23,129.900299,130.451974,127.713291,128.974267,63814900,-0.002798,0,1
2022-12-27,128.097488,129.456984,126.806960,129.427431,69007800,-0.013878,0,1


In [ ]:
dataInst = Data_Storage('AAPL', '2020-01-01', '2023-01-01')
dataInst.manipulate_data()
dataInst.set_update_weight_equal_split_number_to_data(5)
Increment_inst = Get_Incremental_Data(dataInst)
sliding_window_inst = sliding_window(dataInst, Increment_inst, 8)

sliding_window_inst.get_total_weight_and_avg_return_for_all_pattern()


[*********************100%***********************]  1 of 1 completed


Stopping at pattern lenght: 8 as 2^8 > 248


{'0': {'total_avg_return': np.float64(-0.016092092507916704),
  'total_count_weight': np.int64(1077),
  'count': 367},
 '1': {'total_avg_return': np.float64(0.017213417383369168),
  'total_count_weight': np.int64(1184),
  'count': 388},
 '01': {'total_avg_return': np.float64(0.018788301843029053),
  'total_count_weight': np.int64(622),
  'count': 200},
 '10': {'total_avg_return': np.float64(-0.0158213282066599),
  'total_count_weight': np.int64(618),
  'count': 199},
 '11': {'total_avg_return': np.float64(0.01553800838373097),
  'total_count_weight': np.int64(562),
  'count': 188},
 '00': {'total_avg_return': np.float64(-0.01645288280107254),
  'total_count_weight': np.int64(454),
  'count': 167},
 '010': {'total_avg_return': np.float64(-0.01803752000887452),
  'total_count_weight': np.int64(339),
  'count': 103},
 '101': {'total_avg_return': np.float64(0.017256093800414745),
  'total_count_weight': np.int64(351),
  'count': 106},
 '011': {'total_avg_return': np.float64(0.0154491907611